In [16]:
import numpy as np

def lu_decomposition(PA):
    L = np.zeros((m, m))
    U = PA.copy()
    E = np.zeros((m, m))
    
    I = np.zeros((m, m))
    for q in range(m):
        I[q][q] = 1
    
    for j in range(0,n-1):
        #หาค่าU
        for i in range(j+1,m):
            if U[i,j] != 0.0:
                lam = U[i,j]/U[j,j]
                U[i,j:n] = U[i,j:n] - lam*U[j,j:n]
                
                #หาค่าE
                Enew = I.copy()
                for q in range(m):
                    E[q][q] = 1
                Enew[i,j] = -lam
                for y in range(m):
                    for x in range(m):
                        E[y][x] = sum((Enew[y][z] * E[z][x]) for z in range(m))
        #หาค่าL
    try:
        L = np.linalg.inv(E)
    except np.linalg.LinAlgError:
        print("Cannot compute the inverse of the matrix E. It may be singular or non-square.")
        
        # for i in range(j,m):
        #     if i == j:
        #         L[j][j] = 1  
        #     else:
        #         L[i][j] = (PA[i][j] - sum(L[i][k]*U[k][j] for k in range(i))) / U[j][j]
    return U,E,L

def forward_substitution(L, b):
    n = len(b)
    C = np.zeros_like(b)
    for i in range(n):
        C[i] = b[i] - np.dot(L[i, :i], C[:i])
    return C

def backward_substitution(U, C):
    m = len(U)
    n = len(U[0])
    x = [0] * n
    y = C.copy()
    
    free_vars = []
    vars = []
    
    # การแทนค่าผลลัพธ์ย้อนกลับ
    for i in range(m-1, -1, -1):
        k = 0
        while (abs(U[i][k]) < 1e-12):
            k += 1
            if k == n:
                break
        print(k)
        sum = y[i]
        # for k in range(n-1, m-1, -1):
        #     if (abs(U[i][k]) != 0) :
        #         if C[i] == 0:
        #             x[k] = 1  # กำหนดค่าให้กับตัวแปรอิสระ
        #             free_vars.append(k)
        #         else:
        #             x[k] = sum / U[i][k]
        if k < n:
            for j in range(k+1, n):
                if j not in vars:
                    if C[i] == 0:
                        if U[m-1][j] != 0:
                            x[j] = 0
                            vars.append(j)
                        else:
                            x[j] = 1 
                            free_vars.append(j+1)
                            vars.append(j)
                    else :
                    # if U[m-1][j] != 0:
                    #     x[j] = sum / U[i][j]
                    # else:
                        x[j] = 1
                        free_vars.append(j+1)
                        vars.append(j)
                sum -= U[i][j] * x[j]
            # else:
            #     if j not in vars : 
            #         x[j] = 1
            #         free_vars.append(j+1)
            #     sum -= U[i][j] * x[j]
            
            if abs(U[i][k]) < 1e-12:
                if abs(sum) > 1e-12:
                    raise ValueError("ระบบสมการไม่สอดคล้องกัน ไม่มีคำตอบ")
            else:
                x[k] = sum / U[i][k]
                vars.append(k)
            
        else:
            if abs(C[i]) > 1e-12:
                raise ValueError("Inconsistent")
        
    return x, free_vars

#รับขนาดของเมทริกซ์จากผู้ใช้
m = int(input("Enter the size of the matrix (m x n) m: "))
n = int(input("Enter the size of the matrix (m x n) n: "))


#สร้างเมทริกซ์เปล่า
A = np.zeros((m, n))
b = np.zeros((m,1))
P = np.zeros((m, m))
PA = np.zeros((m, n))
Pb = np.zeros((m,1))

#รับค่าของเมทริกซ์จากผู้ใช้
print("Enter the elements of the matrix row by row (each row should have exactly", n, "elements):")
for i in range(m):
    row = input(f"Enter row {i+1} (separate elements by space): ").split()
    A[i] = [float(x) for x in row]

print("กรอกค่าเวกเตอร์ b:")
for i in range(m):
    b[i] = float(input(f"b[{i+1}]: "))


for q in range(m):
    P[q][q] = 1

#สลับแถว P
#for i in range(n):
#    for k in range(i,m):
#        for j in range(m-k):
#            if A[k][i] == 0 and A[k+j][i] != 0:
#                P[[k,k+j]] = P[[k+j,k]]
#                break

#สร้าง PA
#สลับแถว P
for i in range(m):
    for q in range(m):
        for k in range(n):
            PA[q][k] = sum((P[q][r] * A[r][k]) for r in range(m))
    for j in range(i+1,m):
        maxvalue = abs(PA[i,i])
        if abs(PA[j,i]) > maxvalue:
            P[[i, j]] = P[[j, i]]
            maxrow = j
        elif j == m-1 and abs(PA[j,i]) < maxvalue:
            for s in range(n-i):
                i = i+(s+1)
                for v in range(maxrow+2,m):
                    maxvalue = abs(PA[maxrow+1,i])
                    if abs(PA[v,i]) > maxvalue:
                        P[[i, v]] = P[[v, i]]
                        maxrow = v
                            
#สร้าง Pb        
for i in range(m):
        Pb[i] = sum((P[i][j] * b[j][0]) for j in range(m))


#ทำ LU Factorization
U,E,L = lu_decomposition(PA)
C = forward_substitution(L, Pb)

try:
    x,free_vars = backward_substitution(U, C)
    #แสดงผลลัพธ์
    print("\nMatrix A:")
    print(A)

    print("\nMatrix P:")
    print(P)

    print("\nMatrix PA:")
    print(PA)

    print("\nMatrix L (Lower Triangular Matrix):")
    print(L)

    print("\nMatrix E :")
    print(E)

    print("\nMatrix U (Upper Triangular Matrix):")
    print(U)
    
    print("\nSolution C (Lb=C):")
    for Ci in C:
        print(Ci)

    print("\nSolution x:")
    for xi in x:
        print(xi)
    
    print("\nFree variables: " + " ".join(f"x{free_varsi}" for free_varsi in sorted(set(free_vars))))
    
    
except ValueError as e:
    print("No solution:", e)
    

Enter the size of the matrix (m x n) m: 4
Enter the size of the matrix (m x n) n: 6
Enter the elements of the matrix row by row (each row should have exactly 6 elements):
Enter row 1 (separate elements by space): 0 0 0 0 0 0
Enter row 2 (separate elements by space): 1 4 -8 -2 -3 0
Enter row 3 (separate elements by space): 0 0 0 0 1 -4
Enter row 4 (separate elements by space): 0 1 -7 2 -9 -1
กรอกค่าเวกเตอร์ b:
b[1]: 0
b[2]: -4
b[3]: 2
b[4]: -3
Cannot compute the inverse of the matrix E. It may be singular or non-square.
6
4
1
0

Matrix A:
[[ 0.  0.  0.  0.  0.  0.]
 [ 1.  4. -8. -2. -3.  0.]
 [ 0.  0.  0.  0.  1. -4.]
 [ 0.  1. -7.  2. -9. -1.]]

Matrix P:
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]

Matrix PA:
[[ 1.  4. -8. -2. -3.  0.]
 [ 0.  1. -7.  2. -9. -1.]
 [ 0.  0.  0.  0.  1. -4.]
 [ 0.  0.  0.  0.  0.  0.]]

Matrix L (Lower Triangular Matrix):
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Matrix E :
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]